<a href="https://colab.research.google.com/github/bdanver/machine_learning/blob/master/projects/bach_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Main Process

In [1]:
# import required libraries 
from pandas import Series, DataFrame
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from collections import Counter

In [2]:
# download the zip with our bach dataset 
!wget https://raw.githubusercontent.com/zacharski/ml-class/master/data/bach.zip

--2020-10-20 23:05:31--  https://raw.githubusercontent.com/zacharski/ml-class/master/data/bach.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 41761 (41K) [application/zip]
Saving to: ‘bach.zip.2’

bach.zip.2          100%[===================>]  40.78K  --.-KB/s    in 0.01s   

2020-10-20 23:05:31 (2.99 MB/s) - ‘bach.zip.2’ saved [41761/41761]



In [3]:
# extract the file in the zip 
!unzip bach.zip

Archive:  bach.zip
replace bach.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: bach.csv                


In [4]:
# load the csv file to a dataframe
chorales = pd.read_csv('bach.csv')
chorales

,choral_ID,event_number,C,C#,D,D#,E,F,F#,G,G#,A,A#,B,bass,meter,chord_label
0,000106b_,1,YES,NO,NO,NO,NO,YES,NO,NO,NO,YES,NO,NO,F,3,F_M
1,000106b_,2,YES,NO,NO,NO,YES,NO,NO,YES,NO,NO,NO,NO,E,5,C_M
2,000106b_,3,YES,NO,NO,NO,YES,NO,NO,YES,NO,NO,NO,NO,E,2,C_M
3,000106b_,4,YES,NO,NO,NO,NO,YES,NO,NO,NO,YES,NO,NO,F,3,F_M
4,000106b_,5,YES,NO,NO,NO,NO,YES,NO,NO,NO,YES,NO,NO,F,2,F_M
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5660,015505b_,105,NO,NO,YES,NO,NO,NO,NO,YES,NO,NO,YES,NO,G,4,G_m
5661,015505b_,106,NO,NO,YES,NO,NO,NO,NO,YES,NO,YES,NO,NO,G,3,G_m
5662,015505b_,107,YES,NO,NO,NO,YES,NO,NO,YES,NO,NO,NO,NO,C,5,C_M
5663,015505b_,108,YES,NO,NO,NO,YES,NO,NO,YES,NO,NO,YES,NO,C,3,C_M


In [5]:
# sets the choral_ID column as the index for dataframe 
chorales.set_index('choral_ID', inplace=True)
chorales

,event_number,C,C#,D,D#,E,F,F#,G,G#,A,A#,B,bass,meter,chord_label
choral_ID,,,,,,,,,,,,,,,,
000106b_,1,YES,NO,NO,NO,NO,YES,NO,NO,NO,YES,NO,NO,F,3,F_M
000106b_,2,YES,NO,NO,NO,YES,NO,NO,YES,NO,NO,NO,NO,E,5,C_M
000106b_,3,YES,NO,NO,NO,YES,NO,NO,YES,NO,NO,NO,NO,E,2,C_M
000106b_,4,YES,NO,NO,NO,NO,YES,NO,NO,NO,YES,NO,NO,F,3,F_M
000106b_,5,YES,NO,NO,NO,NO,YES,NO,NO,NO,YES,NO,NO,F,2,F_M
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
015505b_,105,NO,NO,YES,NO,NO,NO,NO,YES,NO,NO,YES,NO,G,4,G_m
015505b_,106,NO,NO,YES,NO,NO,NO,NO,YES,NO,YES,NO,NO,G,3,G_m
015505b_,107,YES,NO,NO,NO,YES,NO,NO,YES,NO,NO,NO,NO,C,5,C_M


In [6]:
# evaluate the 12 note columns and replace accordingly 
chorales.replace(('YES', 'NO'), (1, 0), inplace=True)
chorales

,event_number,C,C#,D,D#,E,F,F#,G,G#,A,A#,B,bass,meter,chord_label
choral_ID,,,,,,,,,,,,,,,,
000106b_,1,1,0,0,0,0,1,0,0,0,1,0,0,F,3,F_M
000106b_,2,1,0,0,0,1,0,0,1,0,0,0,0,E,5,C_M
000106b_,3,1,0,0,0,1,0,0,1,0,0,0,0,E,2,C_M
000106b_,4,1,0,0,0,0,1,0,0,0,1,0,0,F,3,F_M
000106b_,5,1,0,0,0,0,1,0,0,0,1,0,0,F,2,F_M
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
015505b_,105,0,0,1,0,0,0,0,1,0,0,1,0,G,4,G_m
015505b_,106,0,0,1,0,0,0,0,1,0,1,0,0,G,3,G_m
015505b_,107,1,0,0,0,1,0,0,1,0,0,0,0,C,5,C_M


In [7]:
# one hot encode the Bass columns 
one_hot_bass = pd.get_dummies(chorales['bass'], prefix='Bass')
one_hot_bass

,Bass_A,Bass_A#,Bass_Ab,Bass_B,Bass_Bb,Bass_C,Bass_C#,Bass_D,Bass_D#,Bass_Db,Bass_E,Bass_Eb,Bass_F,Bass_F#,Bass_G,Bass_G#
choral_ID,,,,,,,,,,,,,,,,
000106b_,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
000106b_,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
000106b_,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
000106b_,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
000106b_,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
015505b_,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
015505b_,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
015505b_,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


In [8]:
# drop bass column and join to main dataframe 
chorales = chorales.drop('bass', axis=1)
chorales = chorales.join(one_hot_bass)
chorales

,event_number,C,C#,D,D#,E,F,F#,G,G#,A,A#,B,meter,chord_label,Bass_A,Bass_A#,Bass_Ab,Bass_B,Bass_Bb,Bass_C,Bass_C#,Bass_D,Bass_D#,Bass_Db,Bass_E,Bass_Eb,Bass_F,Bass_F#,Bass_G,Bass_G#
choral_ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
000106b_,1,1,0,0,0,0,1,0,0,0,1,0,0,3,F_M,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
000106b_,1,1,0,0,0,0,1,0,0,0,1,0,0,3,F_M,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
000106b_,1,1,0,0,0,0,1,0,0,0,1,0,0,3,F_M,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
000106b_,1,1,0,0,0,0,1,0,0,0,1,0,0,3,F_M,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
000106b_,1,1,0,0,0,0,1,0,0,0,1,0,0,3,F_M,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
015505b_,109,0,0,0,0,0,1,0,0,0,1,0,0,4,F_M,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
015505b_,109,0,0,0,0,0,1,0,0,0,1,0,0,4,F_M,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
015505b_,109,0,0,0,0,0,1,0,0,0,1,0,0,4,F_M,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


In [9]:
# assign specific columns to features and labels 
# divide the data into training and testing sets 
featureCols = list(chorales.columns)
featureCols.remove('chord_label')
chorales_features = chorales[featureCols]
chorales_labels = chorales['chord_label']
bach_train_features, bach_test_features, bach_train_labels, bach_test_labels = train_test_split(chorales_features, chorales_labels, test_size = 0.2, random_state=42)
chorales_features

,event_number,C,C#,D,D#,E,F,F#,G,G#,A,A#,B,meter,Bass_A,Bass_A#,Bass_Ab,Bass_B,Bass_Bb,Bass_C,Bass_C#,Bass_D,Bass_D#,Bass_Db,Bass_E,Bass_Eb,Bass_F,Bass_F#,Bass_G,Bass_G#
choral_ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
000106b_,1,1,0,0,0,0,1,0,0,0,1,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
000106b_,1,1,0,0,0,0,1,0,0,0,1,0,0,3,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
000106b_,1,1,0,0,0,0,1,0,0,0,1,0,0,3,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
000106b_,1,1,0,0,0,0,1,0,0,0,1,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
000106b_,1,1,0,0,0,0,1,0,0,0,1,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
015505b_,109,0,0,0,0,0,1,0,0,0,1,0,0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
015505b_,109,0,0,0,0,0,1,0,0,0,1,0,0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
015505b_,109,0,0,0,0,0,1,0,0,0,1,0,0,4,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


In [10]:
# create the classifier and fit it to training sets 
clf = tree.DecisionTreeClassifier(criterion='entropy')
clf.fit(bach_train_features, bach_train_labels)

# print accuracy values of each fold and the average
scores = cross_val_score(clf, bach_train_features, bach_train_labels, cv=10)
print(scores)
print("The average accuracy is %5.3f" % (scores.mean()))


[0.98635396 0.98660435 0.98585319 0.98731351 0.98702139 0.98635368
 0.98614502 0.98608242 0.98656234 0.98704225]
The average accuracy is 0.987


In [20]:
# evaluate how accurate the classifier is in guessing the correct chord_label 
predictions = clf.predict(bach_test_features)
accuracy_score(bach_test_labels, predictions)

0.9863037708448095

## Original Process
###### This is the original thought process I had but this method had low accuracy scores. 

In [12]:
# read data into a dataframe 
chorales_copy = pd.read_csv('bach.csv')  
chorales_copy.set_index('choral_ID', inplace=True)
chorales_copy

,event_number,C,C#,D,D#,E,F,F#,G,G#,A,A#,B,bass,meter,chord_label
choral_ID,,,,,,,,,,,,,,,,
000106b_,1,YES,NO,NO,NO,NO,YES,NO,NO,NO,YES,NO,NO,F,3,F_M
000106b_,2,YES,NO,NO,NO,YES,NO,NO,YES,NO,NO,NO,NO,E,5,C_M
000106b_,3,YES,NO,NO,NO,YES,NO,NO,YES,NO,NO,NO,NO,E,2,C_M
000106b_,4,YES,NO,NO,NO,NO,YES,NO,NO,NO,YES,NO,NO,F,3,F_M
000106b_,5,YES,NO,NO,NO,NO,YES,NO,NO,NO,YES,NO,NO,F,2,F_M
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
015505b_,105,NO,NO,YES,NO,NO,NO,NO,YES,NO,NO,YES,NO,G,4,G_m
015505b_,106,NO,NO,YES,NO,NO,NO,NO,YES,NO,YES,NO,NO,G,3,G_m
015505b_,107,YES,NO,NO,NO,YES,NO,NO,YES,NO,NO,NO,NO,C,5,C_M


In [13]:
# evaluate the chord_label and bass columns to help get a better understanding of data  
print(chorales_copy.chord_label.unique().size)
print(chorales_copy.chord_label.unique())
print(chorales_copy.bass.unique().size)
print(chorales_copy.bass.unique())

102
['F_M' 'C_M' 'D_m' 'BbM' 'C_M7' 'D_m7' 'G_M' 'A_m' 'C_M4' 'G_m' 'G_M7'
 'D_M' 'F#d' 'AbM' 'C#d7' 'D_M7' 'A_M' 'EbM' 'F_M7' 'C_m' 'A_M4' 'F#m'
 'B_M' 'E_M' 'D#d' 'C#M' 'F#m7' 'A#d' 'B_m' 'E_M7' 'E_m' 'F#M' 'F#M7'
 'B_M7' 'A_m6' 'C#d' 'A_m7' 'A_d' 'C_m7' 'G_m6' 'D_M4' 'G_m7' 'E_m7'
 'E_m6' 'A_M7' 'C#m' 'C#M7' 'A#d7' 'EbM7' 'B_m7' 'E_M4' 'C#m7' 'F_M4'
 'D_m6' 'G#d' 'G#d7' 'A_m4' 'D_M6' 'F_d' 'B_m6' 'B_M4' 'C_m6' 'F_m' 'G_M4'
 'F_m7' 'C_d6' 'F_m6' 'G_M6' 'A_M6' 'F_M6' 'F#M4' 'B_d7' 'D_d7' 'E_d'
 'Bbm' 'Bbm6' 'DbM' 'C_d7' 'F#d7' 'Abd' 'Ebd' 'Dbd7' 'Dbd' 'Bbd' 'Dbm7'
 'Abm' 'DbM7' 'Dbm' 'F#m6' 'G#m' 'B_d' 'C_M6' 'D#m' 'D#M' 'BbM7' 'F_d7'
 'C#d6' 'G_d' 'G#M' 'C#M4' 'D#d6' 'D#d7']
16
['F' 'E' 'D' 'A' 'Bb' 'C' 'G' 'B' 'F#' 'C#' 'Eb' 'D#' 'G#' 'A#' 'Ab' 'Db']


In [14]:
# count how many unique bass notes there are
bass_results = Counter()
chorales_copy['bass'].str.split().apply(bass_results.update)
print("bass: ", len(bass_results.keys()))
print(bass_results)

bass:  16
Counter({'D': 689, 'A': 688, 'G': 681, 'E': 678, 'C': 600, 'F': 462, 'F#': 456, 'B': 449, 'Bb': 289, 'C#': 188, 'Eb': 181, 'G#': 147, 'D#': 70, 'Ab': 34, 'Db': 30, 'A#': 23})


In [15]:
# Original thought process was to assign a value to each Bass note 
# dict is in order from most common to least common bass note 

replace_bass = {'D' : 1, 'A' : 2, 'G' : 3, 'E' : 4, 'C' : 5, 'F' : 6, 'F#' : 7, 'B' : 8, 'Bb' : 9,'C#' : 10,'Eb' : 11, 'G#' : 12, 
                         'D#': 13, 'Ab': 14, 'Db': 15, 'A#': 16}        
       
chorales_copy.replace((replace_bass.keys()), (replace_bass.values()), inplace=True)
chorales_copy

,event_number,C,C#,D,D#,E,F,F#,G,G#,A,A#,B,bass,meter,chord_label
choral_ID,,,,,,,,,,,,,,,,
000106b_,1,YES,NO,NO,NO,NO,YES,NO,NO,NO,YES,NO,NO,6,3,F_M
000106b_,2,YES,NO,NO,NO,YES,NO,NO,YES,NO,NO,NO,NO,4,5,C_M
000106b_,3,YES,NO,NO,NO,YES,NO,NO,YES,NO,NO,NO,NO,4,2,C_M
000106b_,4,YES,NO,NO,NO,NO,YES,NO,NO,NO,YES,NO,NO,6,3,F_M
000106b_,5,YES,NO,NO,NO,NO,YES,NO,NO,NO,YES,NO,NO,6,2,F_M
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
015505b_,105,NO,NO,YES,NO,NO,NO,NO,YES,NO,NO,YES,NO,3,4,G_m
015505b_,106,NO,NO,YES,NO,NO,NO,NO,YES,NO,YES,NO,NO,3,3,G_m
015505b_,107,YES,NO,NO,NO,YES,NO,NO,YES,NO,NO,NO,NO,5,5,C_M


In [16]:
# evaluate the 12 note columns and replace accordingly 
chorales_copy.replace(('YES', 'NO'), (1, 0), inplace=True)
chorales_copy

,event_number,C,C#,D,D#,E,F,F#,G,G#,A,A#,B,bass,meter,chord_label
choral_ID,,,,,,,,,,,,,,,,
000106b_,1,1,0,0,0,0,1,0,0,0,1,0,0,6,3,F_M
000106b_,2,1,0,0,0,1,0,0,1,0,0,0,0,4,5,C_M
000106b_,3,1,0,0,0,1,0,0,1,0,0,0,0,4,2,C_M
000106b_,4,1,0,0,0,0,1,0,0,0,1,0,0,6,3,F_M
000106b_,5,1,0,0,0,0,1,0,0,0,1,0,0,6,2,F_M
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
015505b_,105,0,0,1,0,0,0,0,1,0,0,1,0,3,4,G_m
015505b_,106,0,0,1,0,0,0,0,1,0,1,0,0,3,3,G_m
015505b_,107,1,0,0,0,1,0,0,1,0,0,0,0,5,5,C_M


In [17]:
# assign columns to features and labels 
# divide data into training and testing sets 
features = list(chorales_copy.columns)
features.remove('chord_label')
features_copy = chorales_copy[features]
labels_copy = chorales_copy['chord_label']
bach_train_featuresC, bach_test_featuresC, bach_train_labelsC, bach_test_labelsC = train_test_split(features_copy, labels_copy, test_size = 0.2, random_state=42)

In [18]:
# create the classifier and fit it to training sets 
clf2 = tree.DecisionTreeClassifier(criterion='entropy')
clf2.fit(bach_train_featuresC, bach_train_labelsC)

# print accuracy values of each fold and the average
scores2 = cross_val_score(clf2, bach_train_featuresC, bach_train_labelsC, cv=10)
print(scores2)
print("The average accuracy is %5.3f" % (scores2.mean()))

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  % (min_groups, self.n_splits)), UserWarning)


[0.68942731 0.66740088 0.68432671 0.66887417 0.66445916 0.69536424
 0.70640177 0.67549669 0.66666667 0.70860927]
The average accuracy is 0.683


In [19]:
# evaluate how accurate the classifier is in guessing the correct chord_label
predictions2 = clf2.predict(bach_test_featuresC)
accuracy_score(bach_test_labelsC, predictions2)

0.6646072374227714

So one can assume the score is so low because label encoding the Bass column creates the problem of priority in the training set. The note 'F' is assigned to 6, while 'B' is assigned to 8. This means a B note has a higher priority than
a F note. Also, if you take the average of a F note and B note, ( (6+8) / 2 = 7), you would get a 'F#' note... and that does not really represent how similiar (or not similiar) the notes are. With one hot encoding, the bass feature can be binarized into whether the bass note is a F, B, and so on, allowing for the classifier to have a better 'read' of predicting the correct chord.

I have neither given nor received any unauthorized info on this project. -Bridget Danver